## Overview
The business has been investing in four different display advertising campaigns. You have been asked to evaluate how effective
each advertising campaign is in generating sales. Please see below the explanation of the data and the business questions.

The data on the left include information on 10000 consumers who clicked on at least one of the display ads from campaigns
A, B, C, or D. Purchase is indicated by the "Conversion" variable (i.e., equals 1 if there is purchase and 0 otherwise). The "Value" column indicates the revenue in dollars earned from each purchase. The cost per click is $7, $5, $4, and $2 for campaign A, B, C, and D, respectively. The order of clicks is as indicated in the data.

You are asked to build a statistical model (so not a rule based attribution model such as last click attribution) and answer:

Questions
1. Which campaign is the most successful in terms of unit sales contributed?

2. What is the return on investment for each campaign?

3. How would you optimize the spend of a given budget of $1 million across all four campaigns?

#### Assume the cost of campaign is only charged when there is a conversion
#### The idea is to construct transition probability matrix between campaigns, to evaluate each campaign effect on conversion units

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv("./data/user_click.csv")
columns = ["user", "click_1", "click_2", "click_3", "click_4", "click_5", "conversion", "value"]
df = df[columns]
display(df.head(5))

,user,click_1,click_2,click_3,click_4,click_5,conversion,value
0,1,d,d,a,b,NaN,0,0
1,2,c,NaN,NaN,NaN,NaN,0,0
2,3,b,d,a,b,c,1,44
3,4,b,a,a,NaN,NaN,0,0
4,5,d,b,d,NaN,NaN,0,0


In [3]:
df[["click_1", "click_2", "click_3", "click_4", "click_5"]] = df[["click_1", "click_2", "click_3", "click_4", "click_5"]].fillna("NULL")

#### DEFINE STATES
##### START, a, b, c, d, NULL, Conversion

In [4]:
#The cost per click is $7, $5, $4, and $2 for campaign A, B, C, and D, respectively. 
# The company pay for the cost per click
# the profit = revenue 
cpc_dict = {"a": 7, "b": 5, "c": 4, "d": 2}
df["cost"] = 0
for i in range(1, 6):
    df[f"click_{i}_cost"] = df[f"click_{i}"].map(cpc_dict).fillna(0)
    df["cost"] += df[f"click_{i}_cost"]
df["profit"] = df["value"] - df["cost"]
df.profit[df.profit < 0] = 0
df["path"] = df[["click_1", "click_2", "click_3", "click_4", "click_5"]].astype(str).agg(" ".join, axis=1) #.str.split(" ")

In [5]:
df['path'] = np.where(df['conversion'] == 0,
                      ['Start '] + df['path'] + [' NULL'],
                      ['Start '] + df['path'] + [' Conversion'])
df["path"] = df["path"].str.split(" ")

In [6]:
df.head(5)

,user,click_1,click_2,click_3,click_4,click_5,conversion,value,cost,click_1_cost,click_2_cost,click_3_cost,click_4_cost,click_5_cost,profit,path
0,1,d,d,a,b,NULL,0,0,16.0,2,2.0,7.0,5.0,0.0,0.0,"[Start, d, d, a, b, NULL, NULL]"
1,2,c,NULL,NULL,NULL,NULL,0,0,4.0,4,0.0,0.0,0.0,0.0,0.0,"[Start, c, NULL, NULL, NULL, NULL, NULL]"
2,3,b,d,a,b,c,1,44,23.0,5,2.0,7.0,5.0,4.0,21.0,"[Start, b, d, a, b, c, Conversion]"
3,4,b,a,a,NULL,NULL,0,0,19.0,5,7.0,7.0,0.0,0.0,0.0,"[Start, b, a, a, NULL, NULL, NULL]"
4,5,d,b,d,NULL,NULL,0,0,9.0,2,5.0,2.0,0.0,0.0,0.0,"[Start, d, b, d, NULL, NULL, NULL]"


In [7]:
df.cost.sum(), df.value.sum(), df.profit.sum()

(134066.0, 87934, 50954.0)

In [8]:
list_of_paths = df['path']
total_conversions = sum(path.count('Conversion') for path in df['path'].tolist())
base_conversion_rate = total_conversions / len(list_of_paths)
total_value = df["value"].sum()
base_revenue_rate = total_value / len(list_of_paths)
total_profit = df["profit"].sum()
base_profit_rate = total_profit / len(list_of_paths)

#### BUILD TRANSITION STATES

In [9]:
def transition_states(list_of_paths):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    #print(list_of_unique_channels)
    transition_states = {x + '-->' + y: 0 for x in list_of_unique_channels for y in list_of_unique_channels}
    #print(transition_states)
    for possible_state in list_of_unique_channels:
        if possible_state not in ['Conversion', 'NULL']:
            for user_path in list_of_paths:
                if possible_state in user_path:
                    indices = [i for i, s in enumerate(user_path) if possible_state in s]
                    for col in indices:
                        #print(col)
                        #print(user_path)
                        #print(user_path[col])
                        if col < len(user_path) - 1:
                            transition_states[user_path[col] + '-->' + user_path[col + 1]] += 1

    return transition_states

In [10]:
trans_states = transition_states(list_of_paths)

In [11]:
trans_states

{'Conversion-->Conversion': 0,
 'Conversion-->a': 0,
 'Conversion-->d': 0,
 'Conversion-->c': 0,
 'Conversion-->NULL': 0,
 'Conversion-->b': 0,
 'Conversion-->Start': 0,
 'a-->Conversion': 183,
 'a-->a': 1226,
 'a-->d': 1214,
 'a-->c': 1212,
 'a-->NULL': 2243,
 'a-->b': 1286,
 'a-->Start': 0,
 'd-->Conversion': 157,
 'd-->a': 1219,
 'd-->d': 1223,
 'd-->c': 1212,
 'd-->NULL': 2334,
 'd-->b': 1236,
 'd-->Start': 0,
 'c-->Conversion': 165,
 'c-->a': 1168,
 'c-->d': 1228,
 'c-->c': 1256,
 'c-->NULL': 2356,
 'c-->b': 1256,
 'c-->Start': 0,
 'NULL-->Conversion': 0,
 'NULL-->a': 0,
 'NULL-->d': 0,
 'NULL-->c': 0,
 'NULL-->NULL': 0,
 'NULL-->b': 0,
 'NULL-->Start': 0,
 'b-->Conversion': 192,
 'b-->a': 1258,
 'b-->d': 1259,
 'b-->c': 1252,
 'b-->NULL': 2370,
 'b-->b': 1277,
 'b-->Start': 0,
 'Start-->Conversion': 0,
 'Start-->a': 4986,
 'Start-->d': 3403,
 'Start-->c': 3426,
 'Start-->NULL': 0,
 'Start-->b': 3513,
 'Start-->Start': 0}

#### FROM TRANSITION FREQUENCY, BUILD TRANSITION PROBABILITY MATRIX

In [12]:
def transition_probability(trans_dict):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    trans_prob = defaultdict(dict)
    for state in list_of_unique_channels:
        if state not in ['Conversion', 'NULL']:
            counter = 0
            index = [i for i, s in enumerate(trans_dict) if state + '-->' in s]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    counter += trans_dict[list(trans_dict)[col]]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    state_prob = float((trans_dict[list(trans_dict)[col]])) / float(counter)
                    trans_prob[list(trans_dict)[col]] = state_prob

    return trans_prob

In [13]:
state_probability = transition_probability(trans_states)
pprint(state_probability)

defaultdict(<class 'dict'>,
            {'Start-->a': 0.32528705636743216,
             'Start-->b': 0.2291884133611691,
             'Start-->c': 0.2235125260960334,
             'Start-->d': 0.22201200417536535,
             'a-->Conversion': 0.024850624660510592,
             'a-->NULL': 0.30458989679521997,
             'a-->a': 0.16648560564910375,
             'a-->b': 0.1746333514394351,
             'a-->c': 0.1645844649646931,
             'a-->d': 0.1648560564910375,
             'b-->Conversion': 0.025236593059936908,
             'b-->NULL': 0.3115141955835962,
             'b-->a': 0.16535226077812828,
             'b-->b': 0.16784963196635122,
             'b-->c': 0.16456361724500526,
             'b-->d': 0.16548370136698212,
             'c-->Conversion': 0.02221025710055189,
             'c-->NULL': 0.3171355498721228,
             'c-->a': 0.15722169874814915,
             'c-->b': 0.1690671692017768,
             'c-->c': 0.1690671692017768,
             'c-->d': 0.

In [14]:
def construct_transition_matrix(list_of_paths, state_probability):
    transition_matrix = pd.DataFrame()
    list_of_unique_channels = set(x for element in list_of_paths for x in element)

    for channel in list_of_unique_channels:
        transition_matrix[channel] = 0.00
        transition_matrix.loc[channel] = 0.00
        transition_matrix.loc[channel][channel] = 1.0 if channel in ['Conversion', 'NULL'] else 0.0

    for key, value in state_probability.items():
        origin, destination = key.split('-->')
        transition_matrix.at[origin, destination] = value

    return transition_matrix

transition_matrix = construct_transition_matrix(list_of_paths, state_probability)
print("Markov transition matrix")
display(transition_matrix)

Markov transition matrix


,Conversion,a,d,c,NULL,b,Start
Conversion,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
a,0.024851,0.166486,0.164856,0.164584,0.304590,0.174633,0.0
d,0.021271,0.165154,0.165696,0.164205,0.316217,0.167457,0.0
c,0.022210,0.157222,0.165298,0.169067,0.317136,0.169067,0.0
NULL,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0
b,0.025237,0.165352,0.165484,0.164564,0.311514,0.167850,0.0
Start,0.000000,0.325287,0.222012,0.223513,0.000000,0.229188,0.0


#### HAVING TRANSITION MATRIX, CALCULATE REMOVAL EFFECTS OF EACH CAMPAIGN

In [15]:
def removal_effects(df, conversion_rate):
    removal_effects_dict = {}
    channels = [channel for channel in df.columns if channel not in ['Start',
                                                                     'NULL',
                                                                     'Conversion']]
    for channel in channels:
        removal_df = df.drop(channel, axis=1).drop(channel, axis=0)
        for column in removal_df.columns:
            row_sum = np.sum(list(removal_df.loc[column]))
            null_pct = float(1) - row_sum
            if null_pct != 0:
                removal_df.loc[column]['NULL'] = null_pct
            removal_df.loc['NULL']['NULL'] = 1.0

        removal_to_conv = removal_df[
            ['NULL', 'Conversion']].drop(['NULL', 'Conversion'], axis=0)
        removal_to_non_conv = removal_df.drop(
            ['NULL', 'Conversion'], axis=1).drop(['NULL', 'Conversion'], axis=0)

        removal_inv_diff = np.linalg.inv(
            np.identity(
                len(removal_to_non_conv.columns)) - np.asarray(removal_to_non_conv))
        removal_dot_prod = np.dot(removal_inv_diff, np.asarray(removal_to_conv))
        removal_cvr = pd.DataFrame(removal_dot_prod,
                                   index=removal_to_conv.index)[[1]].loc['Start'].values[0]
        removal_effect = 1 - removal_cvr / conversion_rate
        removal_effects_dict[channel] = removal_effect

    return {k: v for k, v in sorted(removal_effects_dict.items())} 

campaign_removal_effect = removal_effects(transition_matrix, base_conversion_rate)

In [16]:
def allocator(removal_effects, unit):
    sum_effect = sum(list(removal_effects.values()))    
    allocation_result = {k: (v / sum_effect) * unit for k, v in sorted(removal_effects.items())}    
    return allocation_result

conversion_unit_attributions = allocator(campaign_removal_effect, 1)

#### RESULT OF CONVERSTION ATTRIBUTION

In [17]:
removal_effects_dict = removal_effects(transition_matrix, base_conversion_rate)
revenue_attributions = allocator(removal_effects_dict, total_value)
print("removal_effects_dict")
print(removal_effects_dict)
print("Conversion Unit Attribution")
print(conversion_unit_attributions)
print("revenue attributions")
print(revenue_attributions)

removal_effects_dict
{'a': 0.8597100706463602, 'b': 0.8414695477652324, 'c': 0.8312865403541212, 'd': 0.8293959429065695}
Conversion Unit Attribution
{'a': 0.2557243707940063, 'b': 0.25029865066347073, 'c': 0.24726967234635122, 'd': 0.24670730619617173}
revenue attributions
{'a': 22486.86682140015, 'b': 22009.761547441634, 'c': 21743.411368104047, 'd': 21693.960263054167}


#### Cost per campaign
(again, assume the cost is charged when there is conversion)

In [18]:
cost_campaign = {}
for campaign in ["a", "b", "c", "d"]:
    cost_campaign[campaign] = 0
    for n_click in range(1, 6):
        cost_campaign[campaign] += df.loc[((df[f"click_{n_click}"]==campaign)
                                           &(df["value"] > 0)), f"click_{n_click}_cost"].sum()

In [19]:
cost_campaign

{'a': 15470.0, 'b': 10060.0, 'c': 7916.0, 'd': 3542.0}

### ROI per campaign

In [20]:
roi_attributions = {}
for k, v in revenue_attributions.items():
    roi_attributions[k] = v - cost_campaign[k]

In [21]:
roi_attributions

{'a': 7016.866821400148,
 'b': 11949.761547441634,
 'c': 13827.411368104047,
 'd': 18151.960263054167}

In [27]:
for k, v in revenue_attributions.items():
    print(k, round(100 * roi_attributions[k] / revenue_attributions[k], 2), "%")

a 31.2 %
b 54.29 %
c 63.59 %
d 83.67 %


#### Budget Allocation

In [22]:
budget = 1e6
budget_attributions = allocator(removal_effects_dict, budget)

In [23]:
print(budget_attributions)

{'a': 255724.37079400627, 'b': 250298.65066347073, 'c': 247269.67234635123, 'd': 246707.30619617173}


### CONCLUSIONS
- Campaign a is the highest contributor of unit sales
- RoI of each campaign:
   - a ~ 7016 (31.2%)
   - b ~ 11949 (54.3%)
   - c ~ 13827 (63.6%)
   - d ~ 18151 (83.7%)
- For budget allocation, the distribution: a / b / c / d = 25.6 / 25 / 24.7 / 24.7 (in %)